In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import xarray as xr
import pickle
#import h5py
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
#import h5py

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

# If available, print the name of the GPU
if cuda_available:
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"Device count: {torch.cuda.device_count()}")

DATA_DIRECTORY = '/oak/stanford/groups/earlew/yuchen'

def save_dict_to_pickle(data_dict, save_path):
    """
    Saves a dictionary to a specified path using pickle.
    
    Parameters:
    data_dict (dict): The dictionary to save.
    save_path (str): The file path where the dictionary will be saved.
    """
    with open(save_path, 'wb') as file:
        pickle.dump(data_dict, file)

class SeaIceDataset(Dataset):
    def __init__(self, data_directory, configuration, split_array, start_prediction_months, \
                split_type='train', target_shape=(336, 320)):
        self.data_directory = data_directory
        self.configuration = configuration
        self.split_array = split_array
        self.start_prediction_months = start_prediction_months
        self.split_type = split_type
        self.target_shape = target_shape

        # Open the HDF5 files
        self.inputs_file = h5py.File(f"{data_directory}/inputs_{configuration}.h5", 'r')
        self.targets_file = h5py.File(f"{data_directory}/targets.h5", 'r')
        
        self.inputs = self.inputs_file[f"inputs_{configuration}"]
        self.targets = self.targets_file['targets_sea_ice_only']

        self.n_samples, self.n_channels, self.n_y, self.n_x = self.inputs.shape
        
        # Get indices for the specified split type
        self.indices = np.where(self.split_array == split_type)[0]

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        actual_idx = self.indices[idx]
        input_data = self.inputs[actual_idx]
        target_data = self.targets[actual_idx]
        start_prediction_month = self.start_prediction_months[actual_idx]

        # Pad input_data and target_data to the target shape
        pad_y = self.target_shape[0] - self.n_y
        pad_x = self.target_shape[1] - self.n_x
        input_data = np.pad(input_data, ((0, 0), (pad_y//2, pad_y//2), (pad_x//2, pad_x//2)), mode='constant', constant_values=0)
        target_data = np.pad(target_data, ((0, 0), (pad_y//2, pad_y//2), (pad_x//2, pad_x//2)), mode='constant', constant_values=0)

        input_tensor = torch.tensor(input_data, dtype=torch.float32)
        target_tensor = torch.tensor(target_data, dtype=torch.float32)

        # Get the target months for this sample
        target_months = pd.date_range(start=start_prediction_month, end=start_prediction_month + pd.DateOffset(months=5), freq="MS")
        target_months = target_months.month.to_numpy()
        
        return input_tensor, target_tensor, target_months

    def __del__(self):
        self.inputs_file.close()
        self.targets_file.close()

class UNetRes3(nn.Module):
    """
    Builds a UNet of resolution 3 (nomenclature from Williams et al. 2023)
    The resolution is defined as the number of encoder/decoder blocks. The
    number at the end of encoder and decoder blocks denote their depth in 
    the network (thus we have, E1 -> E2 -> E3 -> D3 -> D2 -> D1)
    """

    def __init__(self, in_channels, out_channels, device, spatial_shape=(336, 320), n_channels_factor=1, filter_size=3):
        super(UNetRes3, self).__init__()
        
        self.encoder1 = self.conv_block(in_channels, int(64 * n_channels_factor), filter_size)
        self.encoder2 = self.conv_block(int(64 * n_channels_factor), int(128 * n_channels_factor), filter_size)
        self.encoder3 = self.conv_block(int(128 * n_channels_factor), int(256 * n_channels_factor), filter_size)

        self.bottleneck = self.conv_block(int(256 * n_channels_factor), int(512 * n_channels_factor), filter_size)
        
        self.decoder3_conv = self.conv(int(512 * n_channels_factor), int(256 * n_channels_factor), filter_size)
        self.decoder3_conv_block = self.conv_block(2 * int(256 * n_channels_factor), int(256 * n_channels_factor), filter_size)

        self.decoder2_conv = self.conv(int(256 * n_channels_factor), int(128 * n_channels_factor), filter_size)
        self.decoder2_conv_block = self.conv_block(2 * int(128 * n_channels_factor), int(128 * n_channels_factor), filter_size)

        self.decoder1_conv_1 = self.conv(int(128 * n_channels_factor), int(64 * n_channels_factor), filter_size)
        self.decoder1_conv_2 = self.conv(int(64 * n_channels_factor), int(64 * n_channels_factor), filter_size)
        
        self.final_conv = nn.Conv2d(int(64 * n_channels_factor), out_channels, kernel_size=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

        # Make a land mask tensor that is the same shape as the output tensor
        self.land_mask = self.create_land_mask(spatial_shape).to(device)
    
    def create_land_mask(self, spatial_shape):
        land_mask_npy = ~xr.open_dataset(f"{DATA_DIRECTORY}/NSIDC/land_mask.nc").mask.data
        n_y, n_x = land_mask_npy.shape
        pad_y = spatial_shape[0] - n_y
        pad_x = spatial_shape[1] - n_x
        land_mask_npy = np.pad(land_mask_npy, ((pad_y//2, pad_y//2), (pad_x//2, pad_x//2)), mode='constant', constant_values=0)
        
        return torch.from_numpy(land_mask_npy).unsqueeze(0).repeat(6, 1, 1)
        
    def conv_block(self, in_channels, out_channels, filter_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=filter_size, padding=filter_size//2),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=filter_size, padding=filter_size//2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(out_channels)
        )
    
    def conv(self, in_channels, out_channels, filter_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=filter_size, padding=filter_size//2),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))
        bottleneck = self.bottleneck(self.pool(enc3))
        
        dec3 = self.upsample(bottleneck)
        dec3 = self.decoder3_conv(dec3)
        dec3 = torch.cat((enc3, dec3), dim=1)
        dec3 = self.decoder3_conv_block(dec3)
        
        dec2 = self.upsample(dec3)
        dec2 = self.decoder2_conv(dec2)
        dec2 = torch.cat((enc2, dec2), dim=1)
        dec2 = self.decoder2_conv_block(dec2)
        
        dec1 = self.upsample(dec2)
        dec1 = self.decoder1_conv_1(dec1)
        dec1 = torch.cat((enc1, dec1), dim=1)
        dec1 = self.decoder1_conv_1(dec1)
        dec1 = self.decoder1_conv_2(dec1)
        dec1 = self.decoder1_conv_2(dec1)

        output = torch.sigmoid(self.final_conv(dec1))
        
        # Apply the land mask
        output = output * self.land_mask
        
        return output

class UNetRes4(UNetRes3):
    def __init__(self, in_channels, out_channels, device, spatial_shape=(336, 320), n_channels_factor=1, filter_size=3):
        super(UNetRes4, self).__init__(in_channels, out_channels, device, spatial_shape, n_channels_factor, filter_size)

        self.encoder4 = self.conv_block(int(256 * n_channels_factor), int(512 * n_channels_factor), filter_size)
        self.bottleneck = self.conv_block(int(512 * n_channels_factor), int(1024 * n_channels_factor), filter_size)

        self.decoder4_conv = self.conv(int(1024 * n_channels_factor), int(512 * n_channels_factor), filter_size)
        self.decoder4_conv_block = self.conv_block(2 * int(512 * n_channels_factor), int(512 * n_channels_factor), filter_size)
    
    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))
        enc4 = self.encoder4(self.pool(enc3))
        bottleneck = self.bottleneck(self.pool(enc4))

        dec4 = self.upsample(bottleneck)
        dec4 = self.decoder4_conv(dec4)
        dec4 = torch.cat((enc4, dec4), dim=1)
        dec4 = self.decoder4_conv_block(dec4)
        
        dec3 = self.upsample(dec4)
        dec3 = self.decoder3_conv(dec3)
        dec3 = torch.cat((enc3, dec3), dim=1)
        dec3 = self.decoder3_conv_block(dec3)
        
        dec2 = self.upsample(dec3)
        dec2 = self.decoder2_conv(dec2)
        dec2 = torch.cat((enc2, dec2), dim=1)
        dec2 = self.decoder2_conv_block(dec2)
        
        dec1 = self.upsample(dec2)
        dec1 = self.decoder1_conv_1(dec1)
        dec1 = torch.cat((enc1, dec1), dim=1)
        dec1 = self.decoder1_conv_1(dec1)
        dec1 = self.decoder1_conv_2(dec1)
        dec1 = self.decoder1_conv_2(dec1)

        output = torch.sigmoid(self.final_conv(dec1))
        
        # Apply the land mask
        output = output * self.land_mask
        
        return output

class MaskedMSELoss(nn.Module):
    def __init__(self, use_weights, zero_class_weight=None):
        super(MaskedMSELoss, self).__init__()
        self.ice_mask = xr.open_dataset(f"{DATA_DIRECTORY}/NSIDC/monthly_ice_mask.nc").mask
        self.use_weights = use_weights
        self.zero_class_weight = zero_class_weight

    def forward(self, outputs, targets, target_months):
        n_active_cells = 0

        for target_months_subset in target_months:
            n_active_cells += self.ice_mask.sel(month=target_months_subset.cpu()).sum().values
        
        # Punish predictions of sea ice in ice free zones 
        if self.use_weights:
            weights = torch.where(targets == 0, self.zero_class_weight, 1)
            loss = torch.sum(((targets - outputs) ** 2) * weights) / n_active_cells
        else:
            loss = torch.sum((targets - outputs) ** 2) / n_active_cells

        return loss


TRAIN_MONTHS = pd.date_range(start='1981-01-01', end='2014-12-01', freq='MS')
VAL_MONTHS = pd.date_range(start='2015-01-01', end='2018-12-01', freq='MS')
TEST_MONTHS = pd.date_range(start='2019-01-01', end='2024-06-01', freq='MS')

def print_split_stats(split_array):
    ntrain = sum(split_array == 'train')
    nval = sum(split_array == 'val')
    ntest = sum(split_array == 'test')
    
    print(f"train samples: {ntrain} ({round(ntrain / len(split_array), 2)})")
    print(f"val samples: {nval} ({round(nval / len(split_array), 2)})")
    print(f"test samples: {ntest} ({round(ntest / len(split_array), 2)})")

def generate_start_prediction_months(max_month_lead_time=6, max_input_lag_time=12):
    # Construct the date range for the data pairs 
    # Note that this is not continuous due to the missing data in 1987-1988 
    first_range = pd.date_range('1981-01', pd.Timestamp('1987-12') - pd.DateOffset(months=max_month_lead_time+1), freq='MS')
    second_range = pd.date_range(pd.Timestamp('1988-01') + pd.DateOffset(months=max_input_lag_time+1), '2024-01', freq='MS')

    return first_range.append(second_range)


def generate_split_array(verbose=1):
    start_prediction_months = generate_start_prediction_months()
    split_array = np.empty(np.shape(start_prediction_months), dtype=object)
    
    for i,month in enumerate(start_prediction_months):
        if month in TRAIN_MONTHS: split_array[i] = "train"
        if month in VAL_MONTHS: split_array[i] = "val"
        if month in TEST_MONTHS: split_array[i] = "test"

    if verbose == 2: print_split_stats(split_array)
    
    return split_array, start_prediction_months
    


model_hyperparam_configs = {
    "name": "UNetRes3_simpleinputs_b1lr1",
    "architecture": "UNetRes3",
    "input_config": "simple", 
    "batch_size": 4,
    "lr": 1e-4,
    "optimizer": "adam",
    "use_zeros_weight": False,
    "zeros_weight": None,
    "early_stopping": True,
    "patience": 10,
    "max_epochs": 100,
    "notes": "Testing the model training function and early stopping. \
        Similar to UNetRes3_no_zeros_weight but different batch_size."
}

in_channels_config = {
    "sea_ice_only": 14,
    "simple": 23,
    "all": 40
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNetRes3(in_channels=in_channels_config[model_hyperparam_configs["input_config"]], \
                out_channels=6, device=device, n_channels_factor=1, filter_size=3).to(device)

criterion = MaskedMSELoss(use_weights=model_hyperparam_configs["use_zeros_weight"], \
                          zero_class_weight=model_hyperparam_configs["zeros_weight"])

if model_hyperparam_configs["optimizer"] == "adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=model_hyperparam_configs["lr"])
else: 
    raise ValueError("Haven't yet configured training procedure for other optimizers")


In [ ]:
import h5py

split_array, start_prediction_months = generate_split_array()
data_directory = os.path.join(DATA_DIRECTORY, 'sicpred/data_pairs_npy')
configuration = 'simple'
batch_size = 4

# Create dataset instances for training, validation, and testing
train_dataset = SeaIceDataset(data_directory, configuration, split_array, start_prediction_months, split_type='train', target_shape=(336, 320))
val_dataset = SeaIceDataset(data_directory, configuration, split_array, start_prediction_months, split_type='val', target_shape=(336, 320))
test_dataset = SeaIceDataset(data_directory, configuration, split_array, start_prediction_months, split_type='test', target_shape=(336, 320))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)


In [ ]:
def train_model(model, device, model_hyperparam_configs, 
                plot_training_curve=True, verbose=1, save_val_predictions=True):

    # read in some metadata
    configuration = model_hyperparam_configs["input_config"]
    batch_size = model_hyperparam_configs["batch_size"]
    early_stopping = model_hyperparam_configs["early_stopping"]
    patience = model_hyperparam_configs["patience"]
    model_name = model_hyperparam_configs["name"]
    num_epochs = model_hyperparam_configs["max_epochs"]

    # generate array labeling which months are train/val/test
    split_array, start_prediction_months = generate_split_array()

    # create dataset instances for training, validation, and testing
    data_pairs_directory = os.path.join(DATA_DIRECTORY, 'sicpred/data_pairs_npy')
    train_dataset = SeaIceDataset(data_pairs_directory, configuration, split_array, start_prediction_months, split_type='train', target_shape=(336, 320))
    val_dataset = SeaIceDataset(data_pairs_directory, configuration, split_array, start_prediction_months, split_type='val', target_shape=(336, 320))
    test_dataset = SeaIceDataset(data_pairs_directory, configuration, split_array, start_prediction_months, split_type='test', target_shape=(336, 320))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

    train_losses = []
    val_losses = []
    val_predictions = []
    best_val_loss = float('inf')
    epoch_at_early_stopping = None
    patience_counter = 0
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets, target_months in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets, target_months)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)
        if verbose >= 1: print(f"Epoch [{epoch+1}/{num_epochs}], train loss: {epoch_loss:.4f}", end=', ')

        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets, target_months in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets, target_months)
                val_loss += loss.item() * inputs.size(0)        

        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        if verbose >= 1: print(f"validation loss: {val_loss:.4f}")

        # Early stopping check
        if early_stopping:
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # Save the best model weights
                os.makedirs(f"{DATA_DIRECTORY}/sicpred/models", exist_ok=True)
                torch.save(model.state_dict(), f"{DATA_DIRECTORY}/sicpred/models/{model_name}.pth")
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    if verbose >= 1: print("Early stopping triggered")
                    epoch_at_early_stopping = epoch
                    break
        
    # Plot training curves
    if plot_training_curve:
        plt.figure(figsize=(6, 3))
        plt.plot(np.arange(0.5, len(train_losses) + 0.5, 1), train_losses, label="train")
        plt.plot(np.arange(1, len(val_losses) + 1, 1), val_losses, label="val")
        plt.legend()
        plt.title(model_name)

        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.grid()

        os.makedirs(f"{DATA_DIRECTORY}/figures/training_curves", exist_ok=True)
        plt.savefig(f"{DATA_DIRECTORY}/figures/training_curves/{model_name}_training_curve.png", bbox_inches='tight', dpi=300)
        plt.close()

    # Save the validation predictions for hyperparam tuning 
    if save_val_predictions:
        model.eval()
        with torch.no_grad():
            for inputs, targets, target_months in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                val_predictions.append(outputs.cpu().numpy())
        os.makedirs(f"{DATA_DIRECTORY}/sicpred/val_predictions", exist_ok=True)
        np.save(f"{DATA_DIRECTORY}/sicpred/val_predictions/{model_name}_val_predictions.npy", np.concatenate(val_predictions, axis=0))

    # Save the trained model
    if verbose >= 1: print("Saving model weights...", end='')

    # record the best val loss and, if early stopping happened, the last epoch
    model_hyperparam_configs["best_val_loss"] = best_val_loss

    if epoch_at_early_stopping is not None: 
        model_hyperparam_configs["final_epoch"] = epoch_at_early_stopping
    else:
        model_hyperparam_configs["final_epoch"] = num_epochs
        
    os.makedirs(f"{DATA_DIRECTORY}/sicpred/val_predictions", exist_ok=True)
    save_dict_to_pickle(model_hyperparam_configs, f"{DATA_DIRECTORY}/sicpred/models/{model_name}.pkl")
    torch.save(model.state_dict(), f"{DATA_DIRECTORY}/sicpred/models/{model_name}.pth")
    if verbose >= 1: print("done! \n\n")

    return train_losses, val_losses

train_model(model, device, model_hyperparam_configs)

In [ ]:
def load_dict_from_pickle(load_path):
    with open(load_path, 'rb') as file:
        data_dict = pickle.load(file)
    return data_dict

model_name = f"UNetRes3_simpleinputs_b1lr1"

predictions = np.load(f"{DATA_DIRECTORY}/sicpred/val_predictions/{model_name}_val_predictions.npy")

with pickle

In [ ]:

nsidc_sic = xr.open_dataset(f"{DATA_DIRECTORY}/NSIDC/seaice_conc_monthly_all.nc")

model_state_path = os.path.join(f"{DATA_DIRECTORY}/sicpred/models/{model_full_name}.pth")

model.load_state_dict(torch.load(model_state_path, map_location=device))

model.eval()
test_loss = 0.0
inputs_list = []
outputs_list = []
targets_list = []

with torch.no_grad():
    for inputs, targets, target_months in val_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        inputs_list.append(inputs)
        outputs_list.append(outputs)
        targets_list.append(targets)
        loss = criterion(outputs, targets, target_months)
        print(loss.item() * inputs.size(0))


In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12.5, 5))

month_index = 0
lead_time = 0
list_elem = 7

ax1.contourf(nsidc_sic.xgrid, nsidc_sic.ygrid, outputs_list[list_elem][month_index,lead_time,2:334,2:318].cpu(), cmap='nipy_spectral',\
    levels=np.arange(0,1.025,0.025))
cax = ax2.contourf(nsidc_sic.xgrid, nsidc_sic.ygrid, targets_list[list_elem][month_index,lead_time,2:334,2:318].cpu(), cmap='nipy_spectral',\
    levels=np.arange(0,1.025,0.025))

ax1.set_title("Prediction")
ax2.set_title("Truth")

cbar = fig.colorbar(cax, ax=[ax1, ax2], orientation='vertical')
cbar.set_label('Sea ice concentration')

predicted_month = VAL_MONTHS[month_index + lead_time + list_elem*batch_size]

plt.suptitle(rf"Prediction for {predicted_month.month:02}/{predicted_month.year} at {lead_time+1} lead month", fontsize=16)
#plt.savefig(f"../figures/sample_predictions/{model_full_name}_{predicted_month.month:02}_{predicted_month.year}_lead_{lead_time+1}.jpg", dpi=300, bbox_inches='tight')